# OBJECTIVE:
Predict the carbon emissions and energy consumption. 

# KEY FEATURES:

*   ***Entity:*** The name of the country or region for which the data is reported (RELEVANT TO THE INVESTIGATION).

*   ***Year:*** The year for which the data is reported, ranging from 2000 to 2020 (RELEVANT TO THE INVESTIGATION).

*   ***Access to electricity (% of population):*** The percentage of population with access to electricity (RELEVANT TO THE ELECTRICITY FORCASTING).

*   ***Access to clean fuels for cooking (% of population):*** The percentage of the population with primary reliance on clean fuels (RELEVANT TO THE INVESTIGATION).

*   ***Renewable-electricity-generating-capacity-per-capita:*** Installed Renewable energy capacity per person

*   ***Financial flows to developing countries (US $):*** Aid and assistance from developed countries for clean energy projects.

*   ***Renewable energy share in total final energy consumption (%):*** Percentage of renewable energy in final energy consumption.

*   ***Electricity from fossil fuels (TWh):*** Electricity generated from fossil fuels (coal, oil, gas) in terawatt-hours.

*   ***Electricity from nuclear (TWh):*** Electricity generated from nuclear power in terawatt-hours.

*   ***Electricity from renewables (TWh):*** Electricity generated from renewable sources (hydro, solar, wind, etc.) in terawatt-hours.

*   ***Low-carbon electricity (% electricity):*** Percentage of electricity from low-carbon sources (nuclear and renewables).

*   ***Primary energy consumption per capita (kWh/person) -- (TARGET):*** Energy consumption per person in kilowatt-hours.

*   ***Energy intensity level of primary energy (MJ/$2011 PPP GDP):*** Energy use per unit of GDP at purchasing power parity.

*   ***Value_co2_emissions (metric tons per capita) -- (TARGET):*** Carbon dioxide emissions per person in metric tons.

*   ***Renewables (% equivalent primary energy):*** Equivalent primary energy that is derived from renewable sources.

*   ***GDP growth (annual %):*** Annual GDP growth rate based on constant local currency.

*   ***GDP per capita:*** Gross domestic product per person.

*   ***Density (P/Km2):*** Population density in persons per square kilometer.

*   ***Land Area (Km2):*** Total land area in square kilometers.

*   ***Latitude:*** Latitude of the country's centroid in decimal degrees.

*   ***Longitude:*** Longitude of the country's centroid in decimal degrees.

##### NOTE: In parentesis and mayus, we have the variable importance for our investigation (2 targets).

In [1]:
# LIBRARIES:
import pandas as pd
import numpy as np

# IMPUTERS:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# SHOWCASTING THE DATA:
pd.set_option("display.max_column", None)

# TIME SERIES MODELS:
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import grangercausalitytests, adfuller

In [2]:
def NaNs_calculator(data):
    
    '''SHOWING THE STATS OF MISSING DATA AND DATA TYPE'''

    percentage_missing = (data.isna().mean()*100).sort_values(ascending = False)        # Storing the Percentages of NaNs
    sum_missing = data.isna().sum().sort_values(ascending = False)                      # Storing the Sum of NaNs
    names = sum_missing.index.to_list()                                                 # Storing names (to show in the columns)
    data_type = data[names].dtypes                                                      # Storing the type of data based on the order from the previous obtained data (slicing)
    sum_values = sum_missing.to_list()                                                  # Getting count of missing values
    perc_values = np.around(percentage_missing.to_list(), 3)                            # Getting percentage of missing values
    types = data_type.to_list()                                                         # Getting the types of the data
    # TURN ALL THE DATA INTO A DATAFRAME
    df_missing = pd.DataFrame({"NAMES" : names,                                         
                                    "VALUE COUNT" : sum_values,
                                    "PERCENTAGE (%)" : perc_values,
                                    "DATA TYPE": types})
    return df_missing

In [63]:
# IMPORTING DATA:
df = pd.read_csv(r"/Users/alberto/Downloads/PROJECTS/MACHINE-LEARNING-AND-DATA-SCIENCE-PROJECTS/Machine Learning on Cloud/global-data-on-sustainable-energy.csv", parse_dates=['Year'], index_col='Year')
df.head()

,Entity,Access to electricity (% of population),Access to clean fuels for cooking,Renewable-electricity-generating-capacity-per-capita,Financial flows to developing countries (US $),Renewable energy share in the total final energy consumption (%),Electricity from fossil fuels (TWh),Electricity from nuclear (TWh),Electricity from renewables (TWh),Low-carbon electricity (% electricity),Primary energy consumption per capita (kWh/person),Energy intensity level of primary energy (MJ/$2017 PPP GDP),Value_co2_emissions_kt_by_country,Renewables (% equivalent primary energy),gdp_growth,gdp_per_capita,Density\n(P/Km2),Land Area(Km2),Latitude,Longitude
Year,,,,,,,,,,,,,,,,,,,,
2000-01-01,Afghanistan,1.613591,6.2,9.22,20000.0,44.99,0.16,0.0,0.31,65.957440,302.59482,1.64,760.000000,NaN,NaN,NaN,60,"6,52,230",33.93911,67.709953
2001-01-01,Afghanistan,4.074574,7.2,8.86,130000.0,45.60,0.09,0.0,0.50,84.745766,236.89185,1.74,730.000000,NaN,NaN,NaN,60,"6,52,230",33.93911,67.709953
2002-01-01,Afghanistan,9.409158,8.2,8.47,3950000.0,37.83,0.13,0.0,0.56,81.159424,210.86215,1.40,1029.999971,NaN,NaN,179.426579,60,"6,52,230",33.93911,67.709953
2003-01-01,Afghanistan,14.738506,9.5,8.09,25970000.0,36.66,0.31,0.0,0.63,67.021280,229.96822,1.40,1220.000029,NaN,8.832278,190.683814,60,"6,52,230",33.93911,67.709953
2004-01-01,Afghanistan,20.064968,10.9,7.75,NaN,44.24,0.33,0.0,0.56,62.921350,204.23125,1.20,1029.999971,NaN,1.414118,211.382074,60,"6,52,230",33.93911,67.709953


To avoid error when handling the data in columns, the spaces on the columns are removed now

In [64]:
# FORMATTING THE COLUMN NAMES SO IT IS EASIER TO HANDLE THEM (PUTTING THE '_' ON ANY SPACE):
for x in list(df.columns):
    # TESTING STORING AND CHANGING NAMES ACCORDINGLY:
    if(x == "Value_co2_emissions_kt_by_country"):
        name = x.replace("Value_co2_emissions_kt_by_country", "TARGET_2_VALUE_CO2_EMISSIONS_PER_COUNTRY")
        df.rename(columns={x: name}, inplace=True)
    elif(x == "Primary energy consumption per capita (kWh/person)"):
        name = x.replace("Primary energy consumption per capita (kWh/person)", "TARGET_1_PRIMARY_ENERGY_CONSUMPTION_PER_CAPITA_(KWH/PERSON)")
        df.rename(columns={x: name}, inplace=True)
    else:
        if(" " in x):
            name = x.replace(" ", '_')                  
        elif("-" in x):
            name = x.replace("-", '_')                  
        elif("\\n" in x):
            name = x.replace("\\n", '_')
        else:
            name = x                  
        df.rename(columns={x: name.title()}, inplace=True)  # Setting the new name to the column
    name = ''
df.head()

,Entity,Access_To_Electricity_(%_Of_Population),Access_To_Clean_Fuels_For_Cooking,Renewable_Electricity_Generating_Capacity_Per_Capita,Financial_Flows_To_Developing_Countries_(Us_$),Renewable_Energy_Share_In_The_Total_Final_Energy_Consumption_(%),Electricity_From_Fossil_Fuels_(Twh),Electricity_From_Nuclear_(Twh),Electricity_From_Renewables_(Twh),Low-Carbon_Electricity_(%_Electricity),TARGET_1_PRIMARY_ENERGY_CONSUMPTION_PER_CAPITA_(KWH/PERSON),Energy_Intensity_Level_Of_Primary_Energy_(Mj/$2017_Ppp_Gdp),TARGET_2_VALUE_CO2_EMISSIONS_PER_COUNTRY,Renewables_(%_Equivalent_Primary_Energy),Gdp_Growth,Gdp_Per_Capita,Density_(P/Km2),Land_Area(Km2),Latitude,Longitude
Year,,,,,,,,,,,,,,,,,,,,
2000-01-01,Afghanistan,1.613591,6.2,9.22,20000.0,44.99,0.16,0.0,0.31,65.957440,302.59482,1.64,760.000000,NaN,NaN,NaN,60,"6,52,230",33.93911,67.709953
2001-01-01,Afghanistan,4.074574,7.2,8.86,130000.0,45.60,0.09,0.0,0.50,84.745766,236.89185,1.74,730.000000,NaN,NaN,NaN,60,"6,52,230",33.93911,67.709953
2002-01-01,Afghanistan,9.409158,8.2,8.47,3950000.0,37.83,0.13,0.0,0.56,81.159424,210.86215,1.40,1029.999971,NaN,NaN,179.426579,60,"6,52,230",33.93911,67.709953
2003-01-01,Afghanistan,14.738506,9.5,8.09,25970000.0,36.66,0.31,0.0,0.63,67.021280,229.96822,1.40,1220.000029,NaN,8.832278,190.683814,60,"6,52,230",33.93911,67.709953
2004-01-01,Afghanistan,20.064968,10.9,7.75,NaN,44.24,0.33,0.0,0.56,62.921350,204.23125,1.20,1029.999971,NaN,1.414118,211.382074,60,"6,52,230",33.93911,67.709953


In [65]:
# SIZE OF THE DATA:
print(f'The number of rows is: {df.shape[0]}')
print(f'The number of columns is: {df.shape[1]}')

The number of rows is: 3649
The number of columns is: 20


In [41]:
# # CHECKING THE DATA STATISTIC OF THE DATA: (This goes after we remove the NaN values)
# df.describe(include='all').transpose()

It is noticeable that there are a lot of missing values. Let's see how many of them are there per column and the corresponding data type:

In [66]:
# MISSING DATA ON THE DATASET (TOTAL): (This goes before the describe)
df_missing_values = NaNs_calculator(df)
df_missing_values

,NAMES,VALUE COUNT,PERCENTAGE (%),DATA TYPE
0,Renewables_(%_Equivalent_Primary_Energy),2137,58.564,float64
1,Financial_Flows_To_Developing_Countries_(Us_$),2089,57.249,float64
2,Renewable_Electricity_Generating_Capacity_Per_...,931,25.514,float64
3,TARGET_2_VALUE_CO2_EMISSIONS_PER_COUNTRY,428,11.729,float64
4,Gdp_Growth,317,8.687,float64
5,Gdp_Per_Capita,282,7.728,float64
6,Energy_Intensity_Level_Of_Primary_Energy_(Mj/$...,207,5.673,float64
7,Renewable_Energy_Share_In_The_Total_Final_Ener...,194,5.317,float64
8,Access_To_Clean_Fuels_For_Cooking,169,4.631,float64
9,Electricity_From_Nuclear_(Twh),126,3.453,float64


As it can be seen here, there are three variables that are not numbers (Density (P/Km2), Land Area(Km2) and Entity), where two of them are numeric features appears.  The formatting can be devided into four scenarios initially. Later on, based on analysis we can determine what else to do: 

1. Due to a high proportion of missing values (exceeding 20%) in the initial three columns, it becomes challenging to effectively impute them. Therefore, a practical approach would be to discard these columns and proceed with the remaining ones (PENDING TO SEE THE DOMAIN KNOWLEDGE...).

    1.1. Drop the entire column (Drop the columns with more than 20% of missing data)
    
    1.2. Handle the NaNs (drop nas)

2. The other columns need to be parsed into numbers (they are, but for some reason they show as strings).

3. Entity can be handled later on before the model is trained. It is not a variable that affects the initial analysis. In fact, it helps since it shows the Country to be predicted.

NOTE TO MYSELF:

HANDLING THIS LATER ON. CHECKING IF WE DO THE IMPUTE BASED ON HISTORY OR DROP THE COLUMNS WITH A TONS OF NANs

In [67]:
# THESE ARE THE COLUMNS THAT REPORT A HIGH LEVEL OF NANs, WHICH MEANS, THE ONE TO DROP:
print(f'THESE ARE THE COLUMNS THAT REPORT A HIGH LEVEL OF NaNs, WHICH MEANS, THE ONE TO DROP:\n')
df_missing_values[df_missing_values["PERCENTAGE (%)"] > 20]

THESE ARE THE COLUMNS THAT REPORT A HIGH LEVEL OF NaNs, WHICH MEANS, THE ONE TO DROP:



,NAMES,VALUE COUNT,PERCENTAGE (%),DATA TYPE
0,Renewables_(%_Equivalent_Primary_Energy),2137,58.564,float64
1,Financial_Flows_To_Developing_Countries_(Us_$),2089,57.249,float64
2,Renewable_Electricity_Generating_Capacity_Per_...,931,25.514,float64


In [71]:
# TAKING DOWN THE COLUMNS THAT HAVE A LOT OF MISSING VALUES:
names_todelete = df_missing_values[df_missing_values["PERCENTAGE (%)"] > 20]["NAMES"].tolist()      # Getting the names
print(names_todelete)
# In addition, we can get rid of the columns that are less relevant (for now) to our investigation. Longitud and latitud are not useful for the investigation at the moment.
print("Besides the NaNs, there are other variables that at first sight, don't add up to the investigation. We can see them in the following list:")
print("\n1. gdp_growth.\n2. gdp_per_capita.\n3. Energy intensity level of primary energy (MJ/$2017 PPP GDP\n4. Latitude.\n5. Longitude.")
names_todelete.append("Gdp_Growth")
names_todelete.append("Gdp_Per_Capita")
names_todelete.append("Energy_Intensity_Level_Of_Primary_Energy_(Mj/$2017_Ppp_Gdp)")
names_todelete.append("Latitude")
names_todelete.append("Longitude")

# DROPPING COLUMNS:
print(f'Number of columns before dropping: {len(df.columns)}')
filter_df = df.drop(names_todelete, axis=1)                                                         # to keep original data frame as it is.
# filter_df = 
print(f'Number of columns after droping: {len(filter_df.columns)}')
NaNs_calculator(filter_df)

# # HANDLING THIS LATER ON. CHECKING IF WE DO THE IMPUTE BASED ON HISTORY OR DROP THE COLUMNS WITH A TONS OF NAN
# entity = df.groupby("Entity")
# entity.get_group("Afghanistan") # This is to filter the things I want. I will show only the afganistan info
# df[["Entity", "Renewables (% equivalent primary energy)"]] # done it in another way
# df["Entity"].unique()
# # LET'S DELETE IT AND AFTER THAT, IMPUTE THEM

['Renewables_(%_Equivalent_Primary_Energy)', 'Financial_Flows_To_Developing_Countries_(Us_$)', 'Renewable_Electricity_Generating_Capacity_Per_Capita']
Besides the NaNs, there are other variables that at first sight, don't add up to the investigation. We can see them in the following list:

1. gdp_growth.
2. gdp_per_capita.
3. Energy intensity level of primary energy (MJ/$2017 PPP GDP
4. Latitude.
5. Longitude.
Number of columns before dropping: 20
Number of columns after droping: 12


,NAMES,VALUE COUNT,PERCENTAGE (%),DATA TYPE
0,TARGET_2_VALUE_CO2_EMISSIONS_PER_COUNTRY,428,11.729,float64
1,Renewable_Energy_Share_In_The_Total_Final_Ener...,194,5.317,float64
2,Access_To_Clean_Fuels_For_Cooking,169,4.631,float64
3,Electricity_From_Nuclear_(Twh),126,3.453,float64
4,Low-Carbon_Electricity_(%_Electricity),42,1.151,float64
5,Electricity_From_Fossil_Fuels_(Twh),21,0.576,float64
6,Electricity_From_Renewables_(Twh),21,0.576,float64
7,Access_To_Electricity_(%_Of_Population),10,0.274,float64
8,Land_Area(Km2),1,0.027,object
9,Density_(P/Km2),1,0.027,object


To impute the rest of the values, we need to convert non numerical columns used in the analysis into numbers:

In [73]:
# BEFORE IMPUTING, LET MAKE THE CONVERSION: PARSING COLUMNS TO NUMBERS Density (P/Km2) and Land Area(Km2):

# BEFORE CONVERSTION:
print(f'Checking the data before the conversion:\n')
print(f'{filter_df[['Density_(P/Km2)','Land_Area(Km2)']].dtypes}')

# CONVERSION:
# Case 1: Density (P/Km2) got a weird format (12,46,700). Removing just the commas to parse it into a number.
filter_df[['Density_(P/Km2)','Land_Area(Km2)']] = filter_df[['Density_(P/Km2)','Land_Area(Km2)']].apply(lambda x:(x.str.replace(',', '')))
# Case 2: Parsing it into a number:
filter_df[['Density_(P/Km2)','Land_Area(Km2)']] = filter_df[['Density_(P/Km2)','Land_Area(Km2)']].apply(pd.to_numeric)

# AFTER CONVERSTION:
print(f'\nChecking the data after the conversion:\n')
print(f'{filter_df[['Density_(P/Km2)','Land_Area(Km2)']].dtypes}')

Checking the data before the conversion:

Density_(P/Km2)    object
Land_Area(Km2)     object
dtype: object

Checking the data after the conversion:

Density_(P/Km2)    float64
Land_Area(Km2)     float64
dtype: object


In [74]:
# IMPUTING VALUES WITH THE IMPUTER: GETTING THE COLUMNS WHERE WE WANT TO IMPUTE (NOT THE ONE WITH ZERO NaNs):
to_impute = NaNs_calculator(filter_df)                                              # Basically, I am calling again the data frame to filter it
names_to_impute = to_impute[to_impute["PERCENTAGE (%)"] > 0]["NAMES"].tolist()      # Leaving out the columns that got no NaNs
col = filter_df[names_to_impute]                                                    # Selecting only the column names to be imputed
col
# IMPUTING:
imputer = IterativeImputer(max_iter=10, random_state=0)                             # instantiating the imputer
imputed_col = imputer.fit_transform(col)                                            # Fitting the new data to the columns
filter_df[names_to_impute] = imputed_col                                            # Replace the imputed values back into the original DataFrame
print(f'Number of columns after cleaning: {len(filter_df.columns)}')
NaNs_calculator(filter_df)                                                          # Checking the data

Number of columns after cleaning: 12


,NAMES,VALUE COUNT,PERCENTAGE (%),DATA TYPE
0,Entity,0,0.0,object
1,Access_To_Electricity_(%_Of_Population),0,0.0,float64
2,Access_To_Clean_Fuels_For_Cooking,0,0.0,float64
3,Renewable_Energy_Share_In_The_Total_Final_Ener...,0,0.0,float64
4,Electricity_From_Fossil_Fuels_(Twh),0,0.0,float64
5,Electricity_From_Nuclear_(Twh),0,0.0,float64
6,Electricity_From_Renewables_(Twh),0,0.0,float64
7,Low-Carbon_Electricity_(%_Electricity),0,0.0,float64
8,TARGET_1_PRIMARY_ENERGY_CONSUMPTION_PER_CAPITA...,0,0.0,float64
9,TARGET_2_VALUE_CO2_EMISSIONS_PER_COUNTRY,0,0.0,float64


In [75]:
# SHOWING THE DATA SET AGAIN:
filter_df.head(10) 

,Entity,Access_To_Electricity_(%_Of_Population),Access_To_Clean_Fuels_For_Cooking,Renewable_Energy_Share_In_The_Total_Final_Energy_Consumption_(%),Electricity_From_Fossil_Fuels_(Twh),Electricity_From_Nuclear_(Twh),Electricity_From_Renewables_(Twh),Low-Carbon_Electricity_(%_Electricity),TARGET_1_PRIMARY_ENERGY_CONSUMPTION_PER_CAPITA_(KWH/PERSON),TARGET_2_VALUE_CO2_EMISSIONS_PER_COUNTRY,Density_(P/Km2),Land_Area(Km2)
Year,,,,,,,,,,,,
2000-01-01,Afghanistan,1.613591,6.20,44.99,0.16,0.0,0.31,65.957440,302.59482,760.000000,60.0,652230.0
2001-01-01,Afghanistan,4.074574,7.20,45.60,0.09,0.0,0.50,84.745766,236.89185,730.000000,60.0,652230.0
2002-01-01,Afghanistan,9.409158,8.20,37.83,0.13,0.0,0.56,81.159424,210.86215,1029.999971,60.0,652230.0
2003-01-01,Afghanistan,14.738506,9.50,36.66,0.31,0.0,0.63,67.021280,229.96822,1220.000029,60.0,652230.0
2004-01-01,Afghanistan,20.064968,10.90,44.24,0.33,0.0,0.56,62.921350,204.23125,1029.999971,60.0,652230.0
2005-01-01,Afghanistan,25.390894,12.20,33.88,0.34,0.0,0.59,63.440857,252.06912,1549.999952,60.0,652230.0
2006-01-01,Afghanistan,30.718690,13.85,31.89,0.20,0.0,0.64,76.190475,304.42090,1759.999990,60.0,652230.0
2007-01-01,Afghanistan,36.051010,15.30,28.78,0.20,0.0,0.75,78.947370,354.27990,1769.999981,60.0,652230.0
2008-01-01,Afghanistan,42.400000,16.70,21.17,0.19,0.0,0.54,73.972600,607.83350,3559.999943,60.0,652230.0
